In [1]:
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver import ActionChains
import pandas as pd

First Download Firefox Geckodriver and defining the file path where you've saved it.

(I have tested both  Firefox driver and Chrome driver, while Chrome driver caused some minor path problems, Firefox worked flawless in Windows environment.) 

In [2]:
driver=webdriver.Firefox(executable_path=r'C:\Chrome\geckodriver.exe')
url=str('https://seffaflik.epias.com.tr/transparency/tuketim/gerceklesen-tuketim/gercek-zamanli-tuketim.xhtml')
driver.get(url)
driver.maximize_window()
date_value1='01.07.2020'
date_value2='04.07.2020'

First standard step is assigning Selenium geckodriver path as namely driver.(Usually name prefer in this step is "driver" or "browser")
Then, driver.get() function gets the url whatever user choose.

I need to define input date values which will be entered by my Selenium bot automatically.
In this example I have choosen the date between 01.07.2020-04.07.2020 

In [3]:
try:
    date1=WebDriverWait(driver,15).until(EC.presence_of_element_located((By.ID,'j_idt205:date1_input')))

except:
        driver.quit()

date1.clear();
date1.send_keys(date_value1)
try:
    date2=WebDriverWait(driver,15).until(EC.presence_of_element_located((By.ID,'j_idt205:date2_input')))

except:
        driver.quit()

date2.clear();
date2.send_keys(date_value2)
date2.submit()
time.sleep(15)

driver.find_element_by_name('j_idt205:goster').click()

The code block above, Selenium web bot have entered date values automatically in dateboxes of the Epias website and clicked "uygula" the green button for execution.  

In [4]:
if date_value1==date_value2:
    df_1_page = pd.read_html(driver.page_source)[0]
    print(df_1_page)

else:
    df={}
    # df= pd.DataFrame()
    df[0] = pd.read_html(driver.page_source)[0]
    i = 1
    # df_empty.append(df)
    # print("df_empty is ",df_empty)
    while True:

        next_page_btn = driver.find_elements_by_xpath('//*[@id="j_idt205:dt_paginator_bottom"]/a[3]/span')

        try:

                  time.sleep(10)
                  next_page_btn[0].click()
                  # time.sleep(10)
                  table = driver.find_element_by_xpath('//*[@id="j_idt205:dt"]')
                  table_html = table.get_attribute('innerHTML')
                  df[i] = pd.DataFrame()
                  df[i] = pd.read_html(table_html)[0]
                  i=i+1
        except (TimeoutException, WebDriverException) :

                        print("Last page reached")
                        all_pages_df = pd.concat(df)
                        print(all_pages_df)
                        print(len(all_pages_df))

                        break

Last page reached
           Tarih   Saat  Tüketim Miktarı (MWh)
0 0   01.07.2020  00:00               34.01448
  1   01.07.2020  01:00               32.34750
  2   01.07.2020  02:00               31.08524
  3   01.07.2020  03:00               30.55618
  4   01.07.2020  04:00               29.90986
...          ...    ...                    ...
3 19  04.07.2020  19:00               38.52551
  20  04.07.2020  20:00               39.47513
  21  04.07.2020  21:00               39.73661
  22  04.07.2020  22:00               38.40659
  23  04.07.2020  23:00               36.87939

[96 rows x 3 columns]
96


"Finally, The Selenium bot were clicking next button iteratively('>' near the page numbers butons) as if an input user,and pulling all the page's tables to concat in the single Pandas Dataframe format.  "

If you tend to use this dynamic table datas in different software, 
you can just save it as csv format.